In [1]:
# imprescindible
import pandas as pd
import numpy as np

# to avoid some warnings messages
import warnings
warnings.filterwarnings('ignore')

# to draw some graphs
import seaborn as sns
import matplotlib.pyplot as plt

# set seaborn and matplotlib default theme
sns.set_theme()
_sns_plotting_contex_ = sns.plotting_context()
sns.plotting_context('poster')

# set seaborn and matplotlib style to ...
# plt.style.use('classic')
sns.mpl.rcParams['axes.titlesize'] = 18
sns.mpl.rcParams['axes.labelsize'] = 14

# to use HTML codes within IPpython.display function
from IPython.display import HTML



In [2]:

from sklearn.preprocessing import StandardScaler

from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

### Data

In [3]:
data_raw = pd.read_csv("./data/blogData_train.csv", header=None)
data_raw.drop_duplicates(inplace=True)

In [4]:
data_raw.shape

(49203, 281)

In [5]:
X_train = data_raw.iloc[:,0:280]
y_train = data_raw.iloc[:,-1]

y_train = y_train.apply(lambda v : 0 if v <= 50 else (1 if v <= 200 else 2) )

In [6]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)

In [7]:
RMSE = lambda v: round(np.sqrt(v), 4)

class Model :
    def __init__(self, name, model) :
        self.name = name
        self.model = model
        return    


In [8]:

models = []
models.append( Model('Naive Bayes Gaussian', 
                     GaussianNB() 
                    ) 
             )

models.append( Model('Random Forest Classifier', 
                     RandomForestClassifier(
                         n_estimators=100, 
                         criterion='gini', 
                         max_depth=3, 
                         bootstrap=True, 
                         n_jobs=-1, 
                         max_samples=0.3, 
                         random_state=127
                     ) 
                    ) 
             )

models.append( Model('Gradient Boosting Classifier', 
                     GradientBoostingClassifier(
                         # loss='log_loss', # loss function to optimize ???
                         learning_rate=0.1, # contribution of each tree
                         n_estimators=100, # large values -> better performance
                         subsample=1, #  values < 1.0 -> low variance and high bias
                         criterion='friedman_mse', # measure the quality of a split
                         max_depth=3, # tree depth limits the number of nodes
                         random_state=127
                     ) 
                    ) 
             )


for m in models :
    m.model.fit(X_train, y_train)
    y_pred = m.model.predict(X_train)
    display(HTML('<b>' + m.name + '</b>'))
    print(classification_report(y_train, y_pred, digits=6, target_names=None, output_dict=False))
    


              precision    recall  f1-score   support

           0   1.000000  0.012405  0.024506     47884
           1   0.031151  0.125532  0.049915       940
           2   0.008456  1.000000  0.016770       379

    accuracy                       0.022173     49203
   macro avg   0.346536  0.379312  0.030397     49203
weighted avg   0.973853  0.022173  0.024932     49203



              precision    recall  f1-score   support

           0   0.976737  0.999582  0.988027     47884
           1   0.000000  0.000000  0.000000       940
           2   0.678392  0.356201  0.467128       379

    accuracy                       0.975530     49203
   macro avg   0.551710  0.451928  0.485052     49203
weighted avg   0.955778  0.975530  0.965139     49203



              precision    recall  f1-score   support

           0   0.986898  0.997348  0.992096     47884
           1   0.684418  0.369149  0.479613       940
           2   0.839344  0.675462  0.748538       379

    accuracy                       0.982867     49203
   macro avg   0.836887  0.680653  0.740082     49203
weighted avg   0.979983  0.982867  0.980429     49203



---

In [9]:
import os

In [10]:

filepath = './data/test/'
filelist = [os.path.join(filepath, filename) for filename in os.listdir(filepath) if os.path.isfile(os.path.join(filepath, filename))]


In [11]:

test_raw = pd.DataFrame()

for filename in filelist :
    temp_raw = pd.read_csv(filename, header=None)
    temp_raw.drop_duplicates(inplace=True)
    test_raw = test_raw.append(temp_raw)

test_raw.shape

(7036, 281)

In [15]:
X_test = test_raw.iloc[:,0:280]
y_test = test_raw.iloc[:,-1]

y_test = y_test.apply(lambda v : 0 if v <= 50 else (1 if v <= 200 else 2) )

# using train scaler
X_test = scaler.transform(X_test)

for m in models :
    y_pred = m.model.predict(X_test)
    display(HTML('<b>' + m.name + '</b>'))
    print(classification_report(y_test, y_pred, digits=6, target_names=None, output_dict=False))


              precision    recall  f1-score   support

           0   0.968750  0.009018  0.017870      6875
           1   0.039648  0.155172  0.063158       116
           2   0.006137  0.888889  0.012190        45

    accuracy                       0.017055      7036
   macro avg   0.338178  0.351026  0.031072      7036
weighted avg   0.947276  0.017055  0.018580      7036



              precision    recall  f1-score   support

           0   0.979063  0.999855  0.989349      6875
           1   0.000000  0.000000  0.000000       116
           2   0.800000  0.266667  0.400000        45

    accuracy                       0.978681      7036
   macro avg   0.593021  0.422174  0.463116      7036
weighted avg   0.961776  0.978681  0.969269      7036



              precision    recall  f1-score   support

           0   0.984609  0.995636  0.990092      6875
           1   0.423077  0.189655  0.261905       116
           2   0.562500  0.400000  0.467532        45

    accuracy                       0.978539      7036
   macro avg   0.656729  0.528431  0.573176      7036
weighted avg   0.972651  0.978539  0.974744      7036



---


In [19]:

# esto aplica el/los modelos en cada archivo de test

# for filename in filelist :
#     temp_raw = pd.read_csv(filename, header=None)
#     temp_raw.drop_duplicates(inplace=True)

#     X_test = temp_raw.iloc[:,0:280]
#     y_test = temp_raw.iloc[:,-1]

#     y_test = y_test.apply(lambda v : 0 if v <= 50 else (1 if v <= 200 else 2) )

#     # using train scaler
#     X_test = scaler.transform(X_test)

#     print('==================')
#     for m in models :
#         y_pred = m.model.predict(X_test)
#         display(HTML('<b>' + m.name + '</b>'))
#         print(classification_report(y_test, y_pred, digits=6, target_names=None, output_dict=False))


              precision    recall  f1-score   support

           0   1.000000  0.009091  0.018018       110
           1   0.125000  0.500000  0.200000         2
           2   0.009615  1.000000  0.019048         1

    accuracy                       0.026549       113
   macro avg   0.378205  0.503030  0.079022       113
weighted avg   0.975749  0.026549  0.021248       113



              precision    recall  f1-score   support

           0   0.982143  1.000000  0.990991       110
           1   0.000000  0.000000  0.000000         2
           2   1.000000  1.000000  1.000000         1

    accuracy                       0.982301       113
   macro avg   0.660714  0.666667  0.663664       113
weighted avg   0.964918  0.982301  0.973531       113



              precision    recall  f1-score   support

           0   0.982143  1.000000  0.990991       110
           1   0.000000  0.000000  0.000000         2
           2   1.000000  1.000000  1.000000         1

    accuracy                       0.982301       113
   macro avg   0.660714  0.666667  0.663664       113
weighted avg   0.964918  0.982301  0.973531       113



              precision    recall  f1-score   support

           0   1.000000  0.007937  0.015748       126
           1   0.111111  0.333333  0.166667         3
           2   0.008333  1.000000  0.016529         1

    accuracy                       0.023077       130
   macro avg   0.373148  0.447090  0.066315       130
weighted avg   0.971859  0.023077  0.019237       130



              precision    recall  f1-score   support

           0   0.969231  1.000000  0.984375       126
           1   0.000000  0.000000  0.000000         3
           2   0.000000  0.000000  0.000000         1

    accuracy                       0.969231       130
   macro avg   0.323077  0.333333  0.328125       130
weighted avg   0.939408  0.969231  0.954087       130



...

              precision    recall  f1-score   support

           0   0.994652  1.000000  0.997319       186
           1   0.000000  0.000000  0.000000         1
           2   1.000000  1.000000  1.000000         1

    accuracy                       0.994681       188
   macro avg   0.664884  0.666667  0.665773       188
weighted avg   0.989390  0.994681  0.992028       188



              precision    recall  f1-score   support

           0   0.994565  0.983871  0.989189       186
           1   0.000000  0.000000  0.000000         1
           2   0.000000  0.000000  0.000000         1

    accuracy                       0.973404       188
   macro avg   0.331522  0.327957  0.329730       188
weighted avg   0.983985  0.973404  0.978666       188



              precision    recall  f1-score   support

           0   0.000000  0.000000  0.000000       204
           1   0.000000  0.000000  0.000000         3
           2   0.010101  1.000000  0.020000         2

    accuracy                       0.009569       209
   macro avg   0.003367  0.333333  0.006667       209
weighted avg   0.000097  0.009569  0.000191       209



              precision    recall  f1-score   support

           0   0.980769  1.000000  0.990291       204
           1   0.000000  0.000000  0.000000         3
           2   1.000000  0.500000  0.666667         2

    accuracy                       0.980861       209
   macro avg   0.660256  0.500000  0.552319       209
weighted avg   0.966875  0.980861  0.972980       209



              precision    recall  f1-score   support

           0   0.985149  0.975490  0.980296       204
           1   0.000000  0.000000  0.000000         3
           2   0.333333  0.500000  0.400000         2

    accuracy                       0.956938       209
   macro avg   0.439494  0.491830  0.460099       209
weighted avg   0.964770  0.956938  0.960671       209



              precision    recall  f1-score   support

           0   0.968750  0.009018  0.017870      6875
           1   0.050661  0.142857  0.074797       161
           2   0.000000  0.000000  0.000000         0

    accuracy                       0.012081      7036
   macro avg   0.339804  0.050625  0.030889      7036
weighted avg   0.947742  0.012081  0.019173      7036



              precision    recall  f1-score   support

           0   0.979063  0.999855  0.989349      6875
           1   0.000000  0.000000  0.000000       161
           2   0.000000  0.000000  0.000000         0

    accuracy                       0.976976      7036
   macro avg   0.326354  0.333285  0.329783      7036
weighted avg   0.956660  0.976976  0.966711      7036



              precision    recall  f1-score   support

           0   0.984609  0.995636  0.990092      6875
           1   0.576923  0.186335  0.281690       161
           2   0.000000  0.000000  0.000000         0

    accuracy                       0.977118      7036
   macro avg   0.520511  0.393991  0.423927      7036
weighted avg   0.975280  0.977118  0.973882      7036

